In [79]:
from lib import models, graph, coarsening, utils
import numpy as np
import matplotlib.pyplot as plt
import data_converter as dst
import scipy
%matplotlib inline

In [80]:
curr_data_twitter = dst.get_twitter_data_set()
count_data = len(curr_data_twitter["data"])
print(count_data)
count_labels = len(curr_data_twitter["labels"])
print(count_labels)
assert count_data == count_labels
print(curr_data_twitter["label_values"])
count_value_labels = len(curr_data_twitter["label_values"])

144033
144033
{'-1', '1'}


In [81]:
curr_graphs = curr_data_twitter["data"] 
labels = curr_data_twitter["labels"]
new_graphs = [scipy.sparse.coo_matrix(elem) for elem in curr_graphs]
print(new_graphs[0])

  (0, 1)	0.0003034013
  (2, 1)	0.00035130675
  (2, 3)	0.00016500772


In [82]:
n_train = count_data // 2
n_val = count_data // 10

X_train = new_graphs[:n_train]
X_val   = new_graphs[n_train:n_train+n_val]
X_test  = new_graphs[n_train+n_val:]

y_train = labels[:n_train]
y_val   = labels[n_train:n_train+n_val]
y_test  = labels[n_train+n_val:]

In [83]:
L = [graph.laplacian(A, normalized=True) for A in new_graphs]

In [84]:
params = dict()
params['dir_name']       = 'demo'
params['num_epochs']     = 40
params['batch_size']     = 100
params['eval_frequency'] = 200

# Building blocks.
params['filter']         = 'chebyshev5'
params['brelu']          = 'b1relu'
params['pool']           = 'apool1'

# Number of classes.
C = count_value_labels

# Architecture.
params['F']              = [32, 64]  # Number of graph convolutional filters.
params['K']              = [20, 20]  # Polynomial orders.
params['p']              = [4, 2]    # Pooling sizes.
params['M']              = [512, C]  # Output dimensionality of fully connected layers.

# Optimization.
params['regularization'] = 5e-4
params['dropout']        = 1
params['learning_rate']  = 1e-3
params['decay_rate']     = 0.95
params['momentum']       = 0.9
params['decay_steps']    = n_train / params['batch_size']

In [85]:
model = models.cgcnn(L, **params)
accuracy, loss, t_step = model.fit(X_train, y_train, X_val, y_val)

NN architecture
  input: M_0 = 4
  layer 1: cgconv1
    representation: M_0 * F_1 / p_1 = 4 * 32 / 4 = 32
    weights: F_0 * F_1 * K_1 = 1 * 32 * 20 = 640
    biases: F_1 = 32
  layer 2: cgconv2
    representation: M_1 * F_2 / p_2 = 5 * 64 / 2 = 160
    weights: F_1 * F_2 * K_2 = 32 * 64 * 20 = 40960
    biases: F_2 = 64
  layer 3: fc1
    representation: M_3 = 512
    weights: M_2 * M_3 = 160 * 512 = 81920
    biases: M_3 = 512
  layer 4: logits (softmax)
    representation: M_4 = 2
    weights: M_3 * M_4 = 512 * 2 = 1024
    biases: M_4 = 2


TypeError: Input 'b' of 'SparseTensorDenseMatMul' Op has type float32 that does not match type float64 of argument 'a_values'.

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(accuracy, 'b.-')
ax1.set_ylabel('validation accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(loss, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
print('Time per step: {:.2f} ms'.format(t_step*1000))

In [ ]:
res = model.evaluate(X_test, y_test)
print(res[0])